In [133]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import tensorflow as tf
import os
import matplotlib.pyplot as plt

import sys
import numpy as np

In [134]:
import midi
from pypianoroll import Multitrack, Track 
import pypianoroll as pypiano
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#import ipdb

In [135]:
#Generator 
#rows, cols and channels
#Shape of input array

class DCGAN():
    def __init__(self):
        # Input shape
        #self.img_rows = 28
        #self.img_cols = 28
        #self.img_rows = 16
        #self.img_cols = 640
        self.img_rows = 100
        self.img_cols = 128
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        #model.add(Dense(128 * 160 * 4, activation="relu", input_dim=self.latent_dim))
        #model.add(Reshape((4, 160, 128)))
        model.add(Dense(128 * 32 * 25, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((25, 32, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, X_train, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        #(X_train, _), (_, _) = mnist.load_data()
        print(X_train.shape)
        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            print(imgs.shape)
            print(valid.shape)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)
        

    def save_imgs(self, epoch):
        print ("Save begin!")
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        print (gen_imgs.shape)
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        str = ('C:\\Users\\Elisa Pigeon\\Pictures\\V2\\mnist_%d.png' % epoch)
        print(str)
        fig.savefig(str, format ='png')
        plt.close()
        
    def save_imgs_v2(self, epoch, r, c):
        
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        print ("BONJOURRRRR")
        print (gen_imgs.shape)
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        print (gen_imgs.shape)
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        str = ('C:\\Users\\Elisa Pigeon\\Pictures\\V2\\mnist_HH.png')
        print(str)
        fig.savefig(str, format ='png')
        plt.close()


In [136]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = plt.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    result = np.asarray(images)
    return result

In [137]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])
def load_gray_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = plt.imread(os.path.join(folder,filename))
        img = rgb2gray(img)
        if img is not None:
            images.append(img)
    result = np.asarray(images)
    return result

In [138]:
def load_images_from_MIDI_folder(folder):
    #On va couper le fichier Midi pour avoir un fichier de taille (640, 128)
    images = []
    for filename in os.listdir(foldername):
        multitrack = pypiano.parse(os.path.join(foldername,filename))
        list_tracks = multitrack.tracks
        #print(len(list_tracks))
        for track in list_tracks :
            img = track.pianoroll
            img = img[0:100]
            #print(img.shape)
            if img.shape[0] < 100 or img is not None:
                images.append(img)
    result = np.asarray(images)
    return result

In [139]:
foldername = "C:\\Users\\Elisa Pigeon\\Pictures\\mid2"

In [140]:
Train_fileName = 'C:\\Users\\Elisa Pigeon\\Pictures\\Train'
X_train_v2 = load_gray_images_from_folder(Train_fileName)

In [141]:
X_train_v3 = load_images_from_MIDI_folder(foldername)

In [142]:
print(X_train_v3.__class__)
print(X_train_v3[0].__class__)
print(X_train_v3[1].shape)
print(X_train_v2.shape)
print(X_train_v3.shape)
X_train_v3 = np.asarray(X_train_v3)
print(X_train_v3.shape)
X_train_v4 = X_train_v3[0:20]
print(X_train_v4.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(100, 128)
(21, 16, 640)
(196, 100, 128)
(196, 100, 128)
(20, 100, 128)


In [143]:
#if __name__ == '__main__':
print("hello")
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
print("it's begin")
dcgan = DCGAN()

dcgan.train(X_train_v3, epochs=1500, batch_size=32, save_interval=50)
print("The End")

hello
it's begin
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 50, 64, 32)        320       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 50, 64, 32)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 50, 64, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 25, 32, 64)        18496     
_________________________________________________________________
zero_padding2d_5 (ZeroPaddin (None, 26, 33, 64)        0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 26, 33, 64)        256       
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 26, 33, 64)        0   

42 [D loss: 1.411635, acc.: 17.19%] [G loss: 1.547834]
(32, 100, 128, 1)
(32, 1)
43 [D loss: 1.515289, acc.: 20.31%] [G loss: 1.387667]
(32, 100, 128, 1)
(32, 1)
44 [D loss: 1.446350, acc.: 15.62%] [G loss: 1.189380]
(32, 100, 128, 1)
(32, 1)
45 [D loss: 1.316490, acc.: 25.00%] [G loss: 1.296756]
(32, 100, 128, 1)
(32, 1)
46 [D loss: 1.331833, acc.: 21.88%] [G loss: 1.487851]
(32, 100, 128, 1)
(32, 1)
47 [D loss: 1.358721, acc.: 18.75%] [G loss: 1.273098]
(32, 100, 128, 1)
(32, 1)
48 [D loss: 1.474847, acc.: 15.62%] [G loss: 1.493390]
(32, 100, 128, 1)
(32, 1)
49 [D loss: 1.530576, acc.: 20.31%] [G loss: 1.243439]
(32, 100, 128, 1)
(32, 1)
50 [D loss: 1.431399, acc.: 18.75%] [G loss: 1.453743]
Save begin!
(25, 100, 128, 1)
C:\Users\Elisa Pigeon\Pictures\V2\mnist_50.png
(32, 100, 128, 1)
(32, 1)
51 [D loss: 1.356291, acc.: 23.44%] [G loss: 1.532571]
(32, 100, 128, 1)
(32, 1)
52 [D loss: 1.436461, acc.: 23.44%] [G loss: 1.109210]
(32, 100, 128, 1)
(32, 1)
53 [D loss: 1.255105, acc.: 31.2

(32, 100, 128, 1)
(32, 1)
141 [D loss: 1.178890, acc.: 31.25%] [G loss: 1.593773]
(32, 100, 128, 1)
(32, 1)
142 [D loss: 1.463242, acc.: 18.75%] [G loss: 1.351720]
(32, 100, 128, 1)
(32, 1)
143 [D loss: 1.466163, acc.: 10.94%] [G loss: 1.312770]
(32, 100, 128, 1)
(32, 1)
144 [D loss: 1.116708, acc.: 35.94%] [G loss: 1.004803]
(32, 100, 128, 1)
(32, 1)
145 [D loss: 1.379065, acc.: 25.00%] [G loss: 1.257429]
(32, 100, 128, 1)
(32, 1)
146 [D loss: 1.070932, acc.: 35.94%] [G loss: 1.471726]
(32, 100, 128, 1)
(32, 1)
147 [D loss: 1.212833, acc.: 25.00%] [G loss: 1.558004]
(32, 100, 128, 1)
(32, 1)
148 [D loss: 1.165943, acc.: 28.12%] [G loss: 1.612417]
(32, 100, 128, 1)
(32, 1)
149 [D loss: 1.556630, acc.: 15.62%] [G loss: 1.433610]
(32, 100, 128, 1)
(32, 1)
150 [D loss: 1.242618, acc.: 32.81%] [G loss: 1.287462]
Save begin!
(25, 100, 128, 1)
C:\Users\Elisa Pigeon\Pictures\V2\mnist_150.png
(32, 100, 128, 1)
(32, 1)
151 [D loss: 1.273904, acc.: 21.88%] [G loss: 1.208806]
(32, 100, 128, 1)
(3

(32, 100, 128, 1)
(32, 1)
239 [D loss: 1.300171, acc.: 17.19%] [G loss: 1.350998]
(32, 100, 128, 1)
(32, 1)
240 [D loss: 1.401796, acc.: 15.62%] [G loss: 1.532137]
(32, 100, 128, 1)
(32, 1)
241 [D loss: 1.151185, acc.: 32.81%] [G loss: 1.645294]
(32, 100, 128, 1)
(32, 1)
242 [D loss: 1.082880, acc.: 34.38%] [G loss: 1.123775]
(32, 100, 128, 1)
(32, 1)
243 [D loss: 0.881004, acc.: 53.12%] [G loss: 1.380517]
(32, 100, 128, 1)
(32, 1)
244 [D loss: 1.071706, acc.: 34.38%] [G loss: 1.179582]
(32, 100, 128, 1)
(32, 1)
245 [D loss: 1.097726, acc.: 34.38%] [G loss: 1.567020]
(32, 100, 128, 1)
(32, 1)
246 [D loss: 1.076582, acc.: 40.62%] [G loss: 1.162759]
(32, 100, 128, 1)
(32, 1)
247 [D loss: 1.141976, acc.: 29.69%] [G loss: 1.441356]
(32, 100, 128, 1)
(32, 1)
248 [D loss: 0.848316, acc.: 50.00%] [G loss: 1.141685]
(32, 100, 128, 1)
(32, 1)
249 [D loss: 0.761583, acc.: 59.38%] [G loss: 1.567423]
(32, 100, 128, 1)
(32, 1)
250 [D loss: 0.935281, acc.: 45.31%] [G loss: 1.522399]
Save begin!
(25,

(32, 100, 128, 1)
(32, 1)
337 [D loss: 0.794793, acc.: 50.00%] [G loss: 1.955419]
(32, 100, 128, 1)
(32, 1)
338 [D loss: 0.888185, acc.: 50.00%] [G loss: 1.494869]
(32, 100, 128, 1)
(32, 1)
339 [D loss: 0.829872, acc.: 56.25%] [G loss: 1.666065]
(32, 100, 128, 1)
(32, 1)
340 [D loss: 0.636547, acc.: 68.75%] [G loss: 2.374464]
(32, 100, 128, 1)
(32, 1)
341 [D loss: 0.610722, acc.: 60.94%] [G loss: 2.336910]
(32, 100, 128, 1)
(32, 1)
342 [D loss: 0.693191, acc.: 59.38%] [G loss: 2.509003]
(32, 100, 128, 1)
(32, 1)
343 [D loss: 1.236967, acc.: 28.12%] [G loss: 2.764903]
(32, 100, 128, 1)
(32, 1)
344 [D loss: 1.471606, acc.: 21.88%] [G loss: 1.934967]
(32, 100, 128, 1)
(32, 1)
345 [D loss: 0.531254, acc.: 73.44%] [G loss: 1.267133]
(32, 100, 128, 1)
(32, 1)
346 [D loss: 0.497539, acc.: 75.00%] [G loss: 0.987861]
(32, 100, 128, 1)
(32, 1)
347 [D loss: 0.591614, acc.: 67.19%] [G loss: 2.012743]
(32, 100, 128, 1)
(32, 1)
348 [D loss: 1.127282, acc.: 42.19%] [G loss: 2.369413]
(32, 100, 128, 1

...

(32, 100, 128, 1)
(32, 1)
1022 [D loss: 0.163898, acc.: 92.19%] [G loss: 4.725245]
(32, 100, 128, 1)
(32, 1)
1023 [D loss: 0.136850, acc.: 95.31%] [G loss: 4.105426]
(32, 100, 128, 1)
(32, 1)
1024 [D loss: 0.370553, acc.: 78.12%] [G loss: 2.790691]
(32, 100, 128, 1)
(32, 1)
1025 [D loss: 0.224097, acc.: 93.75%] [G loss: 3.872099]
(32, 100, 128, 1)
(32, 1)
1026 [D loss: 0.635531, acc.: 64.06%] [G loss: 3.374310]
(32, 100, 128, 1)
(32, 1)
1027 [D loss: 0.268268, acc.: 87.50%] [G loss: 4.545551]
(32, 100, 128, 1)
(32, 1)
1028 [D loss: 0.218535, acc.: 93.75%] [G loss: 4.445969]
(32, 100, 128, 1)
(32, 1)
1029 [D loss: 0.391708, acc.: 78.12%] [G loss: 2.679930]
(32, 100, 128, 1)
(32, 1)
1030 [D loss: 0.151878, acc.: 95.31%] [G loss: 4.281457]
(32, 100, 128, 1)
(32, 1)
1031 [D loss: 0.202103, acc.: 93.75%] [G loss: 3.227947]
(32, 100, 128, 1)
(32, 1)
1032 [D loss: 0.141666, acc.: 95.31%] [G loss: 4.527422]
(32, 100, 128, 1)
(32, 1)
1033 [D loss: 0.460996, acc.: 76.56%] [G loss: 2.733927]
(32,

1118 [D loss: 0.555285, acc.: 71.88%] [G loss: 2.187365]
(32, 100, 128, 1)
(32, 1)
1119 [D loss: 0.704497, acc.: 64.06%] [G loss: 5.172221]
(32, 100, 128, 1)
(32, 1)
1120 [D loss: 0.295744, acc.: 85.94%] [G loss: 4.410106]
(32, 100, 128, 1)
(32, 1)
1121 [D loss: 0.210816, acc.: 93.75%] [G loss: 3.314707]
(32, 100, 128, 1)
(32, 1)
1122 [D loss: 0.171078, acc.: 93.75%] [G loss: 3.725232]
(32, 100, 128, 1)
(32, 1)
1123 [D loss: 0.402273, acc.: 76.56%] [G loss: 2.833071]
(32, 100, 128, 1)
(32, 1)
1124 [D loss: 0.287821, acc.: 85.94%] [G loss: 4.486867]
(32, 100, 128, 1)
(32, 1)
1125 [D loss: 0.415916, acc.: 75.00%] [G loss: 2.576658]
(32, 100, 128, 1)
(32, 1)
1126 [D loss: 0.142375, acc.: 93.75%] [G loss: 4.366669]
(32, 100, 128, 1)
(32, 1)
1127 [D loss: 0.213916, acc.: 93.75%] [G loss: 3.591139]
(32, 100, 128, 1)
(32, 1)
1128 [D loss: 0.402018, acc.: 78.12%] [G loss: 4.416054]
(32, 100, 128, 1)
(32, 1)
1129 [D loss: 0.158402, acc.: 95.31%] [G loss: 4.925983]
(32, 100, 128, 1)
(32, 1)
1130

(32, 100, 128, 1)
(32, 1)
1215 [D loss: 0.342912, acc.: 82.81%] [G loss: 3.292866]
(32, 100, 128, 1)
(32, 1)
1216 [D loss: 0.381139, acc.: 82.81%] [G loss: 6.015707]
(32, 100, 128, 1)
(32, 1)
1217 [D loss: 1.246364, acc.: 43.75%] [G loss: 2.199732]
(32, 100, 128, 1)
(32, 1)
1218 [D loss: 0.449977, acc.: 76.56%] [G loss: 5.718280]
(32, 100, 128, 1)
(32, 1)
1219 [D loss: 0.318913, acc.: 76.56%] [G loss: 3.415782]
(32, 100, 128, 1)
(32, 1)
1220 [D loss: 0.240879, acc.: 89.06%] [G loss: 3.927891]
(32, 100, 128, 1)
(32, 1)
1221 [D loss: 0.382548, acc.: 81.25%] [G loss: 4.452518]
(32, 100, 128, 1)
(32, 1)
1222 [D loss: 0.130782, acc.: 95.31%] [G loss: 4.464730]
(32, 100, 128, 1)
(32, 1)
1223 [D loss: 0.298465, acc.: 87.50%] [G loss: 4.196176]
(32, 100, 128, 1)
(32, 1)
1224 [D loss: 0.382048, acc.: 84.38%] [G loss: 3.175179]
(32, 100, 128, 1)
(32, 1)
1225 [D loss: 0.075428, acc.: 98.44%] [G loss: 5.084447]
(32, 100, 128, 1)
(32, 1)
1226 [D loss: 0.185669, acc.: 95.31%] [G loss: 2.578013]
(32,

1311 [D loss: 0.207163, acc.: 89.06%] [G loss: 4.230322]
(32, 100, 128, 1)
(32, 1)
1312 [D loss: 0.392024, acc.: 85.94%] [G loss: 2.858485]
(32, 100, 128, 1)
(32, 1)
1313 [D loss: 0.296113, acc.: 85.94%] [G loss: 3.126498]
(32, 100, 128, 1)
(32, 1)
1314 [D loss: 0.128448, acc.: 98.44%] [G loss: 2.918360]
(32, 100, 128, 1)
(32, 1)
1315 [D loss: 0.638995, acc.: 64.06%] [G loss: 4.161168]
(32, 100, 128, 1)
(32, 1)
1316 [D loss: 0.065359, acc.: 98.44%] [G loss: 6.353001]
(32, 100, 128, 1)
(32, 1)
1317 [D loss: 0.564826, acc.: 65.62%] [G loss: 2.620178]
(32, 100, 128, 1)
(32, 1)
1318 [D loss: 0.091146, acc.: 98.44%] [G loss: 4.021770]
(32, 100, 128, 1)
(32, 1)
1319 [D loss: 0.407183, acc.: 78.12%] [G loss: 3.908796]
(32, 100, 128, 1)
(32, 1)
1320 [D loss: 0.745599, acc.: 57.81%] [G loss: 3.419595]
(32, 100, 128, 1)
(32, 1)
1321 [D loss: 0.224236, acc.: 90.62%] [G loss: 4.934987]
(32, 100, 128, 1)
(32, 1)
1322 [D loss: 1.490961, acc.: 35.94%] [G loss: 3.331308]
(32, 100, 128, 1)
(32, 1)
1323

(32, 100, 128, 1)
(32, 1)
1408 [D loss: 0.660387, acc.: 65.62%] [G loss: 2.792977]
(32, 100, 128, 1)
(32, 1)
1409 [D loss: 0.194870, acc.: 89.06%] [G loss: 4.263715]
(32, 100, 128, 1)
(32, 1)
1410 [D loss: 0.206828, acc.: 93.75%] [G loss: 5.061658]
(32, 100, 128, 1)
(32, 1)
1411 [D loss: 0.224954, acc.: 92.19%] [G loss: 2.548120]
(32, 100, 128, 1)
(32, 1)
1412 [D loss: 0.327813, acc.: 84.38%] [G loss: 3.767294]
(32, 100, 128, 1)
(32, 1)
1413 [D loss: 0.129504, acc.: 96.88%] [G loss: 3.143580]
(32, 100, 128, 1)
(32, 1)
1414 [D loss: 0.409410, acc.: 82.81%] [G loss: 3.974868]
(32, 100, 128, 1)
(32, 1)
1415 [D loss: 0.150447, acc.: 98.44%] [G loss: 4.484338]
(32, 100, 128, 1)
(32, 1)
1416 [D loss: 0.280818, acc.: 84.38%] [G loss: 2.596449]
(32, 100, 128, 1)
(32, 1)
1417 [D loss: 0.125057, acc.: 98.44%] [G loss: 3.267471]
(32, 100, 128, 1)
(32, 1)
1418 [D loss: 0.806371, acc.: 57.81%] [G loss: 3.761582]
(32, 100, 128, 1)
(32, 1)
1419 [D loss: 0.589759, acc.: 65.62%] [G loss: 4.426099]
(32,

In [144]:
r =4
c=1
noise = np.random.normal(0, 1, (r * c, dcgan.latent_dim))
gen_images = dcgan.generator.predict(noise)

gen_imgs = gen_images
#gen_imgs = X_train_v3
print(gen_imgs.shape)
gen_imgs = 0.5 * gen_imgs + 0.5
gen_imgs[0].shape
fig, axs = plt.subplots(r, c)
cnt = 0
plt.imshow(gen_imgs[cnt, :,:,0], cmap='gray')
for i in range(r):
        print(i)
        axs[i].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
        axs[i].axis('off')
        cnt += 1
str = ('C:\\Users\\Elisa Pigeon\\Pictures\\V2\\mnist_HH%d.png' %i)
print(str)
fig.savefig(str, format ='png')




filename = "C:\\Users\\Elisa Pigeon\\Pictures\\mid\\Felicita.mid"
test = pypiano.parse(filename)
list_tracks = test.tracks


gen_imgs = gen_images
#gen_imgs = dcgan.generator.predict(noise)
gen_imgs = gen_imgs[:,:,:,0]
print(gen_imgs.shape)
gen_big_img = gen_imgs[0]
for i in range (1, gen_imgs.shape[0]):
    gen_big_img = np.concatenate((gen_big_img, gen_imgs[i]), axis = 0)
print (gen_big_img.shape)
result_fileName = 'C:\\Users\\Elisa Pigeon\\Pictures\\Result\\test8.mid'
multitrack = Multitrack()

print(gen_imgs.shape)
#gen_img = list_tracks[0]
gen_img = gen_imgs[0]
#print(gen_img.pianoroll.shape)
#gen_img = np.asarray(gen_img.pianoroll)

#testb = np.zeros((1000,128))
#testb[158][0] = 1
#print(testb.shape)
#print (testb)
#gen_img = testb
print(gen_img.shape)
gen_img = (gen_img > 0)
#track = Track(pianoroll = gen_img, name = "result")
track = Track(pianoroll = gen_big_img, name = "result")
track.binarize()
track.pianoroll
multitrack.append_track(track)
pypiano.write(multitrack,result_fileName)

(4, 100, 128, 1)
0
1
2
3
C:\Users\Elisa Pigeon\Pictures\V2\mnist_HH3.png
(4, 100, 128)
(400, 128)
(4, 100, 128)
(100, 128)
